# Periodic Differential Riccati Equation Solver

- Written by shamilmamedov
- Ported to Python (CVXPY) by Drew Hamilton


The solver is the implementation of the algorithm proposed in: 
_Gusev, Sergei V., Anton S. Shiriaev, and Leonid B. Freidovich. "SDP-based approximation of  stabilising solutions for periodic matrix Riccati differential equations." International Journal of Control 89.7 (2016): 1396-1405._

Here is the piece of the abstract that provides high level explanation of the algorithm:
> All previously proposed techniques for solving such equations involve numerical integration of unstable differential equations and consequently fail whenever the period is too large or the coefficients vary too much. Here, a new method for numerical computation of stabilising solutions for matrix differential Riccati equations with periodic coefficients is proposed. Our approach does not involve numerical solution of any differential equations. The approximation for a stabilising solution is found in the form of a trigonometric polynomial,
matrix coefficients of which are found solving a specially constructed finite-dimensional semidefinite programming (SDP) problem. This problem is obtained using maximality property of the stabilising solution of the Riccati equation for the associated Riccati inequality and sampling technique.

The Python version solver requires:
* [CVXPY](https://www.cvxpy.org/), installed with !pip install cvxpy
make sure this is installed, along with sympy, scipy and numpy

The implementation consists of several functions:
1. prde\_solver\_settings(Name, Value, ...) - creates a structure for prde solver
2. solver\_prde(A, B, Q, R, t, opts) - solves prde and returns trigonometric matrix polynomial coefficients
3. get\_prde\_solution(t, Xa, Xb, M, w) - computes solution of the prde and its derivative at times instants t
4. get\_state\_feedback(B, R, X) - computes state feedback matrix

- Take a look at the solver_settings function to see which solvers you can use. MOSEK is probably the best, but SDPA gives identical results and is open-source.

In [1]:
import sympy as sp
import numpy as np
from numpy.linalg import inv
from scipy.linalg import solve_continuous_are
from numpy.linalg import pinv
import cvxpy as cp

# We don't want scientific notation, and 6 decimals is fine:
np.set_printoptions(precision=6, suppress=True)

## If you want to compare output to MATLAB version with YALMIP and sdpt3:

In [2]:
# Little Helper function to compare output with MATLAB
# ----------------------------------------------------
# For 3D arrays (hard to view in tables)
# Quick function to compare outputs of MATLAB code to Python outputs
# You check Matlab : P(1,1,27)
# and in Python    : matlab_compare(P, 1,1,27)

def matlab_compare(A, i, j, k):
    print(A[i-1,j-1,k-1])

# Functions definitions:

In [3]:
def benchmark_system():
    
    # This benchmark example is taken from Gusevs paper
    # of 2015 that is taken from somewhere else

    A_c = np.array([[4, 3],
                    [-4.5, -3.5]])
    
    B_c = np.array([[1, -1]]).T
    
    R_c = 1
    
    Q_c1 = np.array([[9,6],[6,4]]).T
    
    Q_c2 = Q_c1 + np.array([[1, 0],[0, 0]])
    Q_c3 = Q_c1 - np.array([[1, 0],[0, 0]])
    
    # S, E switched places vs. Matlab
    # matlab: care(A,B,Q,R,S,E)
    # scipy: solve_continuous_are(a, b, q, r, e, s, balanced)
    H_c = solve_continuous_are(A_c, B_c, Q_c1, R_c, np.eye((2)), np.zeros((2,1)), balanced=True)
    
    H_c = H_c.T
    
    p11_fcn_expr = lambda t: 1/(1.5 + sp.cos(2*sp.pi*t))*sp.cos(2*sp.pi*t)
    p12_fcn_expr = lambda t: 1/(1.5 + sp.cos(2*sp.pi*t))*sp.sin(2*sp.pi*t)
    p21_fcn_expr = lambda t: 1/(1.5 + sp.cos(2*sp.pi*t))*(-sp.sin(2*sp.pi*t))
    p22_fcn_expr = lambda t: 1/(1.5 + sp.cos(2*sp.pi*t))*sp.cos(2*sp.pi*t)
    
    # Finding derivative of matrix P
    tt = sp.Symbol('tt', real=True)    
    pd11_fcn = sp.lambdify(tt, sp.diff(p11_fcn_expr(tt), tt))
    pd12_fcn = sp.lambdify(tt, sp.diff(p12_fcn_expr(tt), tt))
    pd21_fcn = sp.lambdify(tt, sp.diff(p21_fcn_expr(tt), tt))
    pd22_fcn = sp.lambdify(tt, sp.diff(p22_fcn_expr(tt), tt))
    
    T = 1
    N = 100     #Time instarnces
    dt = T/N
    t = np.linspace(0,T,N+1, endpoint=True)
    
    #constructing P matrix
    P = np.zeros((2,2,N+1))
    Pd = np.zeros((2,2,N+1))
    A = np.zeros((2,2,N+1))
    B = np.zeros((2,1,N+1))
    Q = np.zeros((2,2,N+1))
    R = np.zeros((1,1,N+1))
    H = np.zeros((2,2,N+1))
    for i in range(0, len(t)):
        t_i = t[i];
        P[:,:,i] = np.array([[p11_fcn_expr(t_i), p12_fcn_expr(t_i)],
                             [p21_fcn_expr(t_i), p22_fcn_expr(t_i)]])
        Pd[:,:,i] = np.array([[pd11_fcn(t_i), pd12_fcn(t_i)],
                              [pd21_fcn(t_i), pd22_fcn(t_i)]])
        
        A[:,:,i] = np.linalg.solve(P[:,:,i], (A_c@P[:,:,i])) - np.linalg.solve(P[:,:,i], Pd[:,:,i])
        B[:,:,i] = np.linalg.solve(P[:,:,i], B_c)
        Q[:,:,i] = P[:,:,i].T @ Q_c1 @ P[:,:,i]
        R[:,:,i] = R_c
        H[:,:,i] = P[:,:,i].T @ H_c @ P[:,:,i]
    
    return [A, B, Q, R, H, t]

def prde_solver_settings(pol_deg=25, alpha=0, d=1e+5, solver="SDPA"):
    ''' Initialized with defaults taken from test_prde_solver '''
    # Create prde solver settings
    # 
    # Properties:
    #   pol_deg   degree of trigonometric polynomial that servers as a basis
    #             for the solution of the prde
    #   alpha     guaranteed convergence rate i.e. the eigenvalues of the 
    #             closed loop system are going to be on the left of alpha
    #   d         limit on the solution of prde
    #   solver    solver to be used by YALMIP

    opts = {}
    
    opts["pol_deg"] = pol_deg
    opts["alpha"] = alpha
    opts["d"] = d
    opts["solver"] = solver
    
    return opts

def trigonometric_matrix_polynomial(t_k, Xa, Xb, M, w):
    # Function creates trigonometric matrix polynomial of degree M
    # 
    # Inputs:
    #   Xa        symmetric matrix [n x n x M+1] of coefficints of 
    #             polynomial where n - is number of states of linear
    #             time varying system; M - degree of polynomial
    #   Xb        symmetric matrix [n x n x M] of coefficients
    #   t_k       time instant
    #   M         polynomial degree
    #   w         frequency
    # 
    # Outputs:
    #   xt        symbolics value of polynomial at time instant t_k
    #   xtd       symbolics derivative of polynomial at  -/-    
    
        n= Xa[0].shape[0]
    xt = Xa[0]               # R_{a,0} [Saetre, p.33]
    xtd = np.zeros((n,n))    # derivative of R
    for i in range(1, M+1):
        xt = xt + Xa[i] * np.cos(w*i*t_k) + (Xb[i-1]*np.sin(w*i*t_k))
        xtd = xtd - w*i*Xa[i]*np.sin(w*i*t_k) + w*i*Xb[i-1]*np.cos(w*i*t_k)
        
    return [xt, xtd]
    
def solve_prde(A, B, Q, R, t, opts):
    
    # The function solves Periodic Differential Riccati equation for 
    # continous time systems by approximating solution with matrix
    # trigonometric polynomials. The Periodic Differential Riccati equation
    # is imposed at a chosen time instants as LMI such that its Schur
    # compliment is equal to PDRE. Optimization solves for matrix
    # trigonometric polynomial coeffients. Once the optimizator converges
    # we can compute the solution of PDRE at any times instant, thus
    # state feedback matrix
    # 
    # Gusev, Sergei V., Anton S. Shiriaev, and Leonid B. Freidovich. 
    # "SDP-based approximation of stabilising solutions for periodic matrix 
    # Riccati differential equations." 
    # International Journal of Control 89.7 (2016): 1396-1405.
    M, d, alpha, solver = opts["pol_deg"], opts["d"], opts["alpha"], opts["solver"]
    
    T = t[-1]
    N = len(t)
    w = 2*np.pi/T             # frequency
    n = A.shape[0]            # num_rows
    
    J = 0                     # Objective function
    C = []                    # Constraints
    
    # nxnx(m+1) and nxnxm, but cvxpy doesn't have 3d matrices
    # so we'll build a list of nxn cvxpy matrices
    Xa, Xb = [], []
    
    # sdpvar(n, n, M) yalmip symmetric real matrix
    for i in range(0, M+1):
        Xa.append(cp.Variable((n, n), symmetric=True))
    
    # sdpvar(n, n, M) yalmip symmetric real matrix
    for i in range(0, M):
        Xb.append(cp.Variable((n, n),symmetric=True))   
    constraints = []
    
    for k in range(1, N+1):

        [xt, xtd] = trigonometric_matrix_polynomial(t[k-1], Xa, Xb, M, w)
        
        J = J + cp.trace(xt)
        
        Ct = cp.bmat([
             [xtd + A[:,:,k-1].T@xt + xt@A[:,:,k-1] + Q[:,:,k-1], 
              xt@B[:,:,k-1]],
            
             [B[:,:,k-1].T@xt, 
              R[:,:,k-1]]
        ])
        
        constraints.append(Ct >> 0)   
        
        # Removed in Python version.
        #constraints.append(xt >= -d*np.eye(n))    
        #constraints.append(xt <= d*np.eye(n))
    
    # Same as maximize(J)
    prob = cp.Problem(cp.Minimize(-J), constraints)
    
    if solver=="MOSEK":
        # MOSEK of course works best.
        solver=cp.MOSEK
        prob.solve(solver=solver, verbose=False, mosek_params={ "MSK_DPAR_OPTIMIZER_MAX_TIME" : -1.0 } )
    
    elif solver=="SCS":
        # Works, but to much lower precision.
        print("SCS Solver works with PRDE, but to a much lower precision.")
        print("Consider using SDPA or Mosek")
        solver=cp.SCS
        prob.solve(solver=solver, verbose=True)            
        
    elif solver=="SDPA":
        # SDPA works as well as MOSEK in terms of precision, but maybe slower.
        solver=cp.SDPA
        prob.solve(solver=solver, verbose=True, maxIteration=2000)  
    
    else:
        # Assuming people want to use open-source solvers, we default to SDPA:
        print("Solver specified is not good for the problem.")
        print("Defaulting to SDPA...")
        solver=cp.SDPA
        prob.solve(solver=solver, verbose=True, maxIteration=2000)  
    
    # Print result.
    print("The optimal value of J is", J.value)
    
    return [Xa, Xb] #[X1, X2]

##########################################################################################################
def get_prde_solution(t, X1, X2, M, w):
    # Provides the solution of the prde and its derivative
    # 
    # Inputs:
    #   t       time instants at which the solution is required
    #   X1      cosine coeffs of the trigonometric matrix polynomial
    #   X2      sine coeffs of the trigonometric matrix polynomial
    #   M       degree of the trigonometric matrix polynomial
    #   w       fundamental frequency of the trigonometric matrix polynomial
    #
    #   Outputs:
    #   X       solutions of the prde
    #   Xd      time derivatives of the solutions of the prde

    N = t.shape[0]
    no_states = X1[0].value.shape[0]
    
    # Allocating memory for output matrices
    X = np.zeros((no_states, no_states, N))
    Xd = np.zeros((no_states, no_states, N))

    # get_prde_solution, we have want the numeric results of the optimization,
    # not the symbolic decision variables:
    _X1, _X2 = [],[]
    for _x1 in X1:
        _X1.append(_x1.value)
    for _x2 in X2:
        _X2.append(_x2.value)
    X1, X2 = _X1, _X2
    
    for i in range(1,N+1): #for each time step
        [X[:,:,i-1], Xd[:,:,i-1]] = trigonometric_matrix_polynomial(t[i-1], X1, X2, M, w)
        
    return [X, Xd]

def get_state_feedback(B, R, X):
    # Provides the state feedback matrix
    # 
    # Inputs:
    #   B       input-to-state matrix
    #   R       control
    #   X       solution of the PRDE

    N = X.shape[2]
    no_states = B.shape[0]
    no_cotrols = B.shape[1]
    K = np.zeros((no_cotrols, no_states, N))
    for k in range(1, N+1):
        K[:,:,k-1] = np.linalg.solve(-R[:,:,k-1], B[:,:,k-1].T@X[:,:,k-1])
    
    return K

# Now we test the results on the benchmark problem:

## test_prde_solver.m:

In [4]:
[A, B, Q, R, H, t] = benchmark_system()

opts = prde_solver_settings(pol_deg=25, alpha=0, d=1e+5, solver="SDPA")

print("Starting solve...")

# get trigonometric matrix polynomials coeffs
[Xa, Xb] = solve_prde(A, B, Q, R, t, opts)
[X, _] = get_prde_solution(t, Xa, Xb, opts["pol_deg"], 2*np.pi/t[-1])
nrm = np.zeros((X.shape[2],1))

for i in range(1, X.shape[2]+1):
    nrm[i-1] = np.linalg.norm(X[:,:,i-1] - H[:,:,i-1])
    assert nrm[i-1] < 1e-3, "norm out of range. solution may not be good."

print('\nBenchmark Example passed tests - OK\n')

Starting solve...
                                     CVXPY                                     
                                     v1.3.2                                    
(CVXPY) Aug 11 11:36:05 AM: Your problem has 204 variables, 101 constraints, and 0 parameters.


/home/drew/.local/lib/python3.10/site-packages/cvxpy/problems/problem.py:157: UserWarning: Objective contains too many subexpressions. Consider vectorizing your CVXPY code to speed up compilation.
  warnings.warn("Objective contains too many subexpressions. "


(CVXPY) Aug 11 11:36:06 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Aug 11 11:36:06 AM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Aug 11 11:36:06 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Aug 11 11:36:06 AM: Compiling problem (target solver=SDPA).
(CVXPY) Aug 11 11:36:06 AM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffing -> SDPA
(CVXPY) Aug 11 11:36:06 AM: Applying reduction Dcp2Cone
(CVXPY) Aug 11 11:36:08 AM: Applying reduction CvxAttr2Constr
(CVXPY) Aug 11 11:36:09 AM: Applying reduction ConeMatrixStuffing
(CVXPY) Aug 11 11:36:14 AM: Applying reduction SDPA
(CVXPY) Au

In [5]:
K = get_state_feedback(B,R,H)
print("Feedback controller computed with H from benchmark: ", K)

K2 = get_state_feedback(B,R,X)
print("Feedback controller computed with SDP: ", K2)

Feedback controller computed with H from benchmark:  [[[-2.897056 -2.772256 -2.640476 -2.501562 -2.355326 -2.201538 -2.03993
   -1.870194 -1.691979 -1.504893 -1.308497 -1.102304 -0.885778 -0.658332
   -0.419321 -0.168043  0.096263  0.374423  0.667327  0.975933  1.301264
    1.644409  2.006521  2.388809  2.792524  3.218951  3.669381  4.145075
    4.647228  5.176898  5.734932  6.321854  6.937734  7.582008  8.253265
    8.948986  9.665235 10.396309 11.134364 11.869038 12.587137 13.272434
   13.905692 14.465008 14.926602 15.266101 15.460332 15.489498 15.339473
   15.003844 14.485281 13.795893 12.956406 11.994289 10.941164  9.829964
    8.692336  7.556615  6.446537  5.380682  4.372526  3.430899  2.56069
    1.76363   1.039057  0.3846   -0.203257 -0.728718 -1.196332 -1.610746
   -1.976525 -2.298041 -2.579399 -2.824401 -3.03653  -3.218951 -3.37452
   -3.505802 -3.615092 -3.704433 -3.775645 -3.83034  -3.869944 -3.895717
   -3.908769 -3.910075 -3.90049  -3.880759 -3.851532 -3.813369 -3.766753
 